In [5]:
from sqlalchemy import create_engine
from models import Cryptocurrency, Transaction

from tools import (generate_uri_from_file,
                   get_symbol_to_id_dict,
                   get_id_to_symbol_dict)

In [6]:
def get_id(db, ticker='BTC'):
    '''returns the symbol id of the given ticker'''
    sym_to_id_map = get_symbol_to_id_dict(db)
    ticker_id = sym_to_id_map.get(ticker.upper(), None)  #returns ticker_id if it is found, and None when it does not exist
    return None

---
## Database URI
In this section I'm creating the URI to the database so we can connect more easily . I'm making use of my function `generate_uri_from_file` to take the configuration Yaml file as input and return the URI string.

In [7]:
uri = generate_uri_from_file("db_config.yml")

---
## Using Flask-SQLAlchemy to Read and Write to DB
In this section, I will use the flask-SQLAlchemy package to read and write from and to the database. 

In [8]:
from flask_sqlalchemy import SQLAlchemy
from flask import Flask

In [9]:
db = SQLAlchemy()
app = Flask(__name__)

In [10]:
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = uri

In [11]:
db.init_app(app)

#### Using App Context
We need to make use of Flask's app_context method in order to query the database (for standalone scripts). You will get an error if you don't.

In [12]:
with app.app_context():
    results = db.session.query(Transaction).all()

In [13]:
results

[<Transaction: ticker_id=1, coins=100.000000000, cost basis=3999.00000>,
 <Transaction: ticker_id=1, coins=45.000000000, cost basis=10000.00000>,
 <Transaction: ticker_id=1, coins=1.000000000, cost basis=34000.00000>,
 <Transaction: ticker_id=2, coins=0.123456789, cost basis=350.00000>,
 <Transaction: ticker_id=3, coins=1500.000000000, cost basis=100.00000>]

#### How to Retrieve Conditional Data
Retrieving only data that meets certain conditions. For instance, maybe I want to query the data for only the transactions related to Bitcoin (in this case, I would need to know Bitcoins ticker_id on the "transactions" table. The id is 1.

In [14]:
# Returning only Bitcoin transactions (Bitcoin's ticker_id is 1)
with app.app_context():
    results = db.session.query(Transaction).filter(Transaction.ticker_id==1).all()

In [15]:
results

[<Transaction: ticker_id=1, coins=100.000000000, cost basis=3999.00000>,
 <Transaction: ticker_id=1, coins=45.000000000, cost basis=10000.00000>,
 <Transaction: ticker_id=1, coins=1.000000000, cost basis=34000.00000>]

In [16]:
# Returning only Bitcoin transactions and only those that have more than 2 coins per transaction
with app.app_context():
    results = db.session.query(Transaction).filter(Transaction.ticker_id==1, 
                                                   Transaction.num_coins > 2).all()

In [17]:
results

[<Transaction: ticker_id=1, coins=100.000000000, cost basis=3999.00000>,
 <Transaction: ticker_id=1, coins=45.000000000, cost basis=10000.00000>]

#### Writing to the Database
I will create a new sample transaction and write it to the database. In this example, I will add a new XRP transaction. Therefore, we first need to query the table cryptocurrencies to get the appropriate object. If XRP does not exist on the table yet, it will be added as well.

In [18]:
_coin = 'XRP'

with app.app_context():
    
    #create our new transaction
    new_tx = Transaction(num_coins=1500, cost_basis=1215)  #$1,215 assumes an XRP price of around $0.81 -> 1500 XRP * $0.81 ~= $1215
    
    #---Retrieve coin from DB
    coin_obj = db.session.query(Cryptocurrency).filter(Cryptocurrency.ticker==_coin).first()
    if coin_obj is None:
        coin_obj = Cryptocurrency(_coin) #if it does not exist, we create a new object
        db.session.add(coin_obj)
        db.session.commit()
    
    coin_obj.transactions.append(new_tx)
    db.session.add(new_tx)
    db.session.commit()

You can run the toy application with flask (run `python app.py` in the terminal) to check that the data is indeed added to the database. You can also query the database for XRP transactions following the steps I showed already. Other ways of checking the database include accessing the MySQL terminal or some graphical program where you can see your tables and content.